In [20]:
pip install edge-tts pygame langdetect


Note: you may need to restart the kernel to use updated packages.


In [18]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox
import threading
import asyncio
import edge_tts
import pygame
from langdetect import detect
import os
import re
import tempfile
import fitz  # pour les fichiers PDF (PyMuPDF)
from docx import Document  # pour les fichiers DOCX (python-docx)

class TextReaderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Assistant Avatar")
        self.text_data = ""
        self.voice_map = {
            "fr": "fr-FR-DeniseNeural",
            "en": "en-US-AriaNeural",
            "ar": "ar-EG-SalmaNeural",
            "de": "de-DE-KatjaNeural"
        }
        self.language = tk.StringVar(value="auto")
        self.speed = tk.DoubleVar(value=1.0)
        self.volume = tk.DoubleVar(value=1.0)

        self.audio_file = None
        self.paused = False

        self.build_ui()
        pygame.mixer.init()

    def build_ui(self):
        self.root.configure(bg="#f8e1f4")
        
        style = ttk.Style()
        style.configure("TButton", font=("Segoe UI", 12), padding=6, width=12, relief="flat", background="#ff66b2", foreground="black")
        style.configure("TButton:hover", background="#e6007f")
        style.configure("TLabel", font=("Segoe UI", 11), background="#f8e1f4", foreground="#b30059")
        style.configure("TCombobox", font=("Segoe UI", 11), state="readonly", background="#ff33cc", foreground="black", selectbackground="#ff33cc", selectforeground="black", highlightthickness=0)

        main_frame = ttk.Frame(self.root, padding=20)
        main_frame.pack(expand=True, fill="both")

        button_frame = ttk.Frame(main_frame)
        button_frame.pack(pady=10)

        ttk.Button(button_frame, text="Importer", command=self.load_file).pack(side="left", padx=5)
        ttk.Button(button_frame, text="Lire", command=self.play_text).pack(side="left", padx=5)
        ttk.Button(button_frame, text="Pause", command=self.pause_audio).pack(side="left", padx=5)
        ttk.Button(button_frame, text="Reprendre", command=self.resume_audio).pack(side="left", padx=5)
        ttk.Button(button_frame, text="Rejouer", command=self.restart_audio).pack(side="left", padx=5)

        control_frame = ttk.LabelFrame(main_frame, text="Contrôles audio", padding=10)
        control_frame.pack(pady=10, fill="x")

        ttk.Label(control_frame, text="Vitesse").pack(anchor="w")
        ttk.Scale(control_frame, variable=self.speed, from_=0.5, to=2.0, orient="horizontal").pack(fill="x", padx=10)

        ttk.Label(control_frame, text="Volume").pack(anchor="w", pady=(10, 0))
        ttk.Scale(control_frame, variable=self.volume, from_=0.0, to=1.0, orient="horizontal").pack(fill="x", padx=10)

        lang_frame = ttk.Frame(main_frame)
        lang_frame.pack(pady=5)

        ttk.Label(lang_frame, text="Langue :").pack(side="left", padx=5)
        ttk.Combobox(lang_frame, textvariable=self.language, values=["auto", "fr", "en", "ar", "de"], style="TCombobox").pack(side="left", padx=5, fill="x", expand=True)

    def load_file(self):
        file_path = filedialog.askopenfilename(filetypes=[
            ("Text files", "*.txt"),
            ("PDF files", "*.pdf"),
            ("Word files", "*.docx"),
            ("All files", "*.*"),
        ])
        if not file_path:
            return

        try:
            ext = os.path.splitext(file_path)[1].lower()
            if ext == ".txt":
                with open(file_path, "r", encoding="utf-8") as f:
                    self.text_data = f.read()
            elif ext == ".pdf":
                self.text_data = self.extract_pdf_text(file_path)
            elif ext == ".docx":
                self.text_data = self.extract_docx_text(file_path)
            else:
                messagebox.showerror("Erreur", f"Type de fichier non pris en charge : {ext}")
                return
            print("Texte importé avec succès.")
        except Exception as e:
            messagebox.showerror("Erreur", f"Impossible de lire le fichier : {e}")

    def extract_pdf_text(self, path):
     important_text = ""
     with fitz.open(path) as doc:
         for page in doc:
             blocks = page.get_text("blocks")
             for block in blocks:
                 x0, y0, x1, y1, text = block[:5]
                 text = text.strip()
                 if text:
                    # Ignorer les blocs trop à gauche (calendriers visuels) => seuil à ajuster si besoin
                     if x0 < 200:
                         continue
                    # Ignorer les blocs contenant seulement des numéros ou des initiales de jours
                     if re.fullmatch(r"[\d\sLMMJVSD]*", text, flags=re.IGNORECASE):
                         continue
                    # Garder les blocs qui contiennent des événements
                     if re.search(r'\d{2}/\d{2}/\d{2}', text) or any(keyword in text.lower() for keyword in [
                         "évènement", "vacances", "examens", "masters", "concours", "attestations", "pfe",
                         "soutenances", "révision", "début", "rentrée", "résultats", "affichage", "arrêt des cours"
                     ]):
                         important_text += text + "\n"
     return important_text


    def extract_docx_text(self, path):
        doc = Document(path)
        return "\n".join([para.text for para in doc.paragraphs])

    def get_voice(self):
        try:
            if self.language.get() == "auto":
                lang = detect(self.text_data)
                print(f"Langue détectée : {lang}")
            else:
                lang = self.language.get()
            return self.voice_map.get(lang, "en-US-AriaNeural")
        except:
            return "en-US-AriaNeural"

    def play_text(self):
        if not self.text_data.strip():
            messagebox.showwarning("Attention", "Veuillez d'abord importer un fichier texte.")
            return
        threading.Thread(target=self.generate_and_play).start()

    def generate_and_play(self):
        asyncio.run(self.speak_and_play())

    async def speak_and_play(self):
        voice = self.get_voice()
        speed_value = int((self.speed.get() - 1) * 100)
        speed = f"{speed_value:+d}%"

        if pygame.mixer.music.get_busy():
            pygame.mixer.music.stop()

        try:
            communicate = edge_tts.Communicate(text=self.text_data, voice=voice, rate=speed)
            with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
                await communicate.save(temp_audio.name)
                self.audio_file = temp_audio.name
            print(f"Fichier audio généré : {self.audio_file}")
        except Exception as e:
            messagebox.showerror("Erreur TTS", f"Erreur lors de la synthèse vocale : {e}")
            return

        try:
            pygame.mixer.music.load(self.audio_file)
            pygame.mixer.music.set_volume(self.volume.get())
            pygame.mixer.music.play()
            self.paused = False
            print("Lecture audio démarrée.")
        except Exception as e:
            messagebox.showerror("Erreur audio", f"Impossible de jouer l'audio : {e}")

    def pause_audio(self):
        if pygame.mixer.music.get_busy():
            pygame.mixer.music.pause()
            self.paused = True
            print("Lecture en pause.")

    def resume_audio(self):
        if self.paused:
            pygame.mixer.music.unpause()
            self.paused = False
            print("Lecture reprise.")

    def restart_audio(self):
        if self.audio_file and os.path.exists(self.audio_file):
            pygame.mixer.music.stop()
            pygame.mixer.music.load(self.audio_file)
            pygame.mixer.music.set_volume(self.volume.get())
            pygame.mixer.music.play()
            self.paused = False
            print("Lecture redémarrée depuis le début.")

if __name__ == "__main__":
    root = tk.Tk()
    app = TextReaderApp(root)
    root.mainloop()


Texte importé avec succès.
Langue détectée : fr
Fichier audio généré : C:\Users\dorbe\AppData\Local\Temp\tmpmuhhf4js.mp3
Lecture audio démarrée.
Lecture en pause.
